In [34]:
import pandas as pd
from pandas import TimeSeries
import numpy as np
import math
from bokeh.charts import TimeSeries, output_file, show
import statsmodels.tsa.stattools as stat

In [35]:
start=20000103
end=20150101
period=60
num=10

In [36]:
data,pairs=find_pair(start,period,num)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
def data_process(start,period):
    with open ('C:/mine/Course/9894/project/data/sp_500_daily_prices.csv') as data:
        sp500=pd.read_csv(data,usecols=['date','COMNAM','PRC','RET','ASK','BID'],engine='c')
    date=sorted(list(set(sp500['date'])))
    try:
        date.index(start)
    except:
        raise (("Oops! The start date is not contained in our datesets.  Try again..."))
    try:
        train_start=date[-9*period+date.index(start)]
        date[date.index(start)+period]
    except:
        raise ("Oops! The period is too long.  Try again...")
    sp_recent=sp500[sp500['date']>=train_start]
    sp_recent=sp_recent[sp_recent['date']<=date[date.index(start)+period]]
    
    with open ('C:/mine/Course/9894/project/data/sp_500_daily_return.csv') as return_data:
        sp_ret=pd.read_csv(return_data,index_col=0,engine='c')
    sp_ret=sp_ret.loc[date[-9*period+date.index(start)]:date[period+date.index(start)]]
    return sp_recent,sp_ret,date

In [25]:
def filter_pair(pvalue_key,num):
    try:
        pvalue_key[num-1]
    except:
        raise("Oops! Too large num")
    selected_pair=[]
    stock_list=[]
    i=0
    while i<num:
        for pair in pvalue_key:
            if pair[1][0] not in stock_list and pair[1][1] not in stock_list:
                selected_pair+=[pair[1]]
                stock_list+=[pair[1][0]]
                stock_list+=[pair[1][1]]
                i+=1
    
    return selected_pair

In [26]:
def find_pair(start,period,num):
    sp_recent,sp_ret,date=data_process(start,period)
    error_return=['E','D','C','B','A','']
    sp_ret=sp_ret.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))
    sp_pca=np.array(sp_ret)
    #print(sp_pca)
    cov_company=np.cov(sp_pca)
    cov_company[np.isnan(cov_company)] = 0.0
    eigen_val,eigen_vector=np.linalg.eig(cov_company)
    eigen_vector_max=list(zip(eigen_vector[np.argmax(eigen_val)],sp_ret.columns))
    eigen_vector_max.sort(key=lambda x:x[0],reverse=True)
    eigen_vector_max=list(zip(*eigen_vector_max))
    data=sp_ret[list(eigen_vector_max[1][0:20])]
    pvalue_key=find_cointegrated_pairs(data)
    selected_pair=filter_pair(pvalue_key,num)
    data_inuse=sp_recent[sp_recent['date']>=date[-period+date.index(start)]]
    data_inuse=data_inuse[data_inuse['date']<date[date.index(start)+period]]
    
    #print('find_pair_done')
    return data_inuse,selected_pair

In [27]:
def find_cointegrated_pairs(data):
    n = len(data.columns)
    #score_matrix = np.zeros((n, n))
    pvalue_key=[]
    keys = list(data.columns)
    #pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = np.array(data[keys[i]])
            S2 = np.array(data[keys[j]])
            result = stat.coint(S1, S2)
            #score = result[0]
            pvalue = result[1]
            pvalue_key+=[[pvalue,(keys[i],keys[j])]]
            #pairs.append((keys[i], keys[j]))
    pvalue_key=sorted(pvalue_key,key=lambda x:x[0])
    #print('find_cointegrated_pairs_done')
    return pvalue_key